In [1]:
"""
Created on Tue Aug 13 13:51:40 2024

Recherche des meilleurs hyperparamètres pour application de l'algorithme XGBoost en modélisation globale et segmentée

@author: Thierry ALLEM
"""

"\nCreated on Tue Aug 13 13:51:40 2024\n\nRecherche des meilleurs hyperparamètres pour application de l'algorithme XGBoost en modélisation globale et segmentée\n\n@author: Thierry ALLEM\n"

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor  # Import de XGBoost
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from math import sqrt
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Importation du fichier eco2mix_regional_def préparé, incluant les TCO, TCH, cmax_rte et les populations régionales
# Lecture du fichier CSV
df_blackout_ml = pd.read_csv('df_blackout_ML_pop.csv', sep=';',low_memory=False)
# Lecture du fichier CSV
df_blackout_ml.head()

,region,date_heure,annee,jour_numero,heure,jour_fractionnel,thermique,nucleaire,eolien,solaire,...,nucleaire_p_disp,eolien_p_disp,solaire_p_disp,cap_prod_max_hydraulique,ind_prod_hydraulique,hydraulique_p_disp,cap_prod_max_bioenergies,bioenergies_p_disp,region_p_max,population
0,CENTRE VAL DE LOIRE,2013-01-01 00:30:00,2013,1,00:30:00,1.02,90.0,9085.0,508.0,0.0,...,11630.0,662.0,207.0,91.0,0.0,91.0,41.0,41.0,12788.0,2570548
1,PAYS DE LA LOIRE,2013-01-01 00:30:00,2013,1,00:30:00,1.02,127.0,0.0,182.0,0.0,...,0.0,454.0,385.0,5.0,0.0,5.0,57.0,57.0,2738.0,3660852
2,GRAND EST,2013-01-01 00:30:00,2013,1,00:30:00,1.02,319.0,9137.0,1109.0,0.0,...,10820.0,1803.0,498.0,2280.0,0.0,2280.0,79.0,79.0,18878.0,5552388
3,ILE DE FRANCE,2013-01-01 00:30:00,2013,1,00:30:00,1.02,685.0,0.0,16.0,0.0,...,0.0,37.0,108.0,19.0,0.0,19.0,262.0,262.0,2547.0,11959807
4,OCCITANIE,2013-01-01 00:30:00,2013,1,00:30:00,1.02,78.0,2497.0,367.0,0.0,...,2620.0,861.0,1040.0,5368.0,0.0,5368.0,126.0,126.0,10260.0,5683878


In [4]:
# Filtrage des données pour les années 2013 à 2019
df_blackout_ml_filtered = df_blackout_ml.loc[(df_blackout_ml['annee'] >= 2013) & (df_blackout_ml['annee'] <= 2019)].copy()

In [5]:
# Conversion de la colonne 'date_heure' en datetime
df_blackout_ml_filtered['date_heure'] = pd.to_datetime(df_blackout_ml_filtered['date_heure'], errors='coerce')

# Conversion de l'heure et des minutes en heure décimale
df_blackout_ml_filtered['heure_decimale'] = (df_blackout_ml_filtered['date_heure'].dt.hour +
                                             df_blackout_ml_filtered['date_heure'].dt.minute / 60)

In [6]:
# Encodages cycliques des données temporelles
df_blackout_ml_filtered['heure_decimale_sin'] = np.sin(2 * np.pi * df_blackout_ml_filtered['heure_decimale'] / 24)
df_blackout_ml_filtered['heure_decimale_cos'] = np.cos(2 * np.pi * df_blackout_ml_filtered['heure_decimale'] / 24)

In [7]:
# Fonction pour vérifier si une année est bissextile
def bissextile(annee):
    return annee % 4 == 0 and (annee % 100 != 0 or annee % 400 == 0)

In [8]:
# Ajout d'une colonne pour le nombre total de jours dans l'année
df_blackout_ml_filtered['total_jours_annee'] = df_blackout_ml_filtered['annee'].apply(lambda annee: 366 if bissextile(annee) else 365)

In [9]:
# Encodage cyclique pour 'jour_fractionnel'
df_blackout_ml_filtered['jour_fractionnel_sin'] = (np.sin(2 * np.pi * df_blackout_ml_filtered['jour_fractionnel'] /
                                                          df_blackout_ml_filtered['total_jours_annee']))
df_blackout_ml_filtered['jour_fractionnel_cos'] = (np.cos(2 * np.pi * df_blackout_ml_filtered['jour_fractionnel'] /
                                                          df_blackout_ml_filtered['total_jours_annee']))


In [10]:
# Suppression de la colonne 'total_jours_annee'
df_blackout_ml_filtered = df_blackout_ml_filtered.drop(columns='total_jours_annee')

df_blackout_ml_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1472244 entries, 0 to 1472243
Data columns (total 34 columns):
 #   Column                    Non-Null Count    Dtype         
---  ------                    --------------    -----         
 0   region                    1472244 non-null  object        
 1   date_heure                1472244 non-null  datetime64[ns]
 2   annee                     1472244 non-null  int64         
 3   jour_numero               1472244 non-null  int64         
 4   heure                     1472244 non-null  object        
 5   jour_fractionnel          1472244 non-null  float64       
 6   thermique                 1472244 non-null  float64       
 7   nucleaire                 1472244 non-null  float64       
 8   eolien                    1472244 non-null  float64       
 9   solaire                   1472244 non-null  float64       
 10  hydraulique               1472244 non-null  float64       
 11  bioenergies               1472244 non-null  float64    

In [11]:
# Liste des régions uniques
regions = df_blackout_ml_filtered['region'].unique()

In [12]:
# ----------------- OPTIMISATION DES HYPERPARAMETRES POUR MODELISATION GLOBALE ------------------

# Définition de la grille d'hyperparamètres
param_distributions = {
    'n_estimators': [100, 200, 300, 400],
    'max_depth': [3, 4, 5, 6],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9],
    'gamma': [0, 0.1, 0.2]
}

# Définition des variables explicatives et de la variable cible

features = df_blackout_ml_filtered[['region', 'annee', 'jour_fractionnel_sin', 'jour_fractionnel_cos', 'heure_decimale_sin', 'heure_decimale_cos',
                          'thermique', 'nucleaire', 'eolien', 'solaire', 'hydraulique', 'bioenergies',
                          'pompage', 'stockage_batterie', 'destockage_batterie', 'cap_prod_max_thermique','thermique_p_disp',
                          'cap_prod_max_nucleaire', 'nucleaire_p_disp', 'cap_prod_max_hydraulique', 'hydraulique_p_disp',
                          'cap_prod_max_bioenergies','bioenergies_p_disp',
                          'solaire_p_disp', 'eolien_p_disp', 'population']]


target = df_blackout_ml_filtered['ech_physiques']

# Encodage des variables catégorielles (comme 'region')
features = pd.get_dummies(features, columns=['region'])

# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Standardisation des données
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Initialisation du modèle
xgb = XGBRegressor(random_state=42, objective='reg:squarederror', importance_type='gain')

# Configuration de la recherche d'hyperparamètres
random_search = RandomizedSearchCV(estimator=xgb, param_distributions=param_distributions, 
                                   n_iter=50, cv=3, verbose=2, random_state=42, n_jobs=-1)

# Exécution de la recherche d'hyperparamètres
random_search.fit(X_train, y_train)

# Enregistrement des meilleurs hyperparamètres
best_params_global = random_search.best_params_

# Prédictions avec les meilleurs hyperparamètres

y_pred = random_search.best_estimator_.predict(X_test)

# Calcul des métriques
mse = mean_squared_error(y_test, y_pred)
rmse = sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
train_score = random_search.best_estimator_.score(X_train, y_train)
test_score = random_search.best_estimator_.score(X_test, y_test)

# Création d'un DataFrame pour les résultats globaux
results_global = pd.DataFrame({
        'TOP paramètres': [best_params_global],
        'MSE': [mse], 
        'RMSE': [rmse], 
        'MAE': [mae], 
        'R2': [r2],
        'Train Score': [train_score],
        'Test Score': [test_score]
    })

# Exportation des résultats globaux dans un fichier Excel
results_global.to_excel("hyperp_global_xgboost_bruts.xlsx", index=False, engine='openpyxl')

print("Recherche des hyperparamètres globaux terminée et résultats sauvegardés.")

Fitting 3 folds for each of 50 candidates, totalling 150 fits
Recherche des hyperparamètres globaux terminée et résultats sauvegardés.


In [13]:
# ----------------- OPTIMISATION DES HYPERPARAMETRES POUR MODELISATION PAR SEGMENTATION REGIONALE ------------------


In [14]:
# Dictionnaire pour stocker les meilleurs paramètres par région
best_params_region = {}

# Création d'un DataFrame pour stocker les résultats régionaux
results_region = pd.DataFrame(columns=['Region', 'Best Params', 'MSE', 'RMSE', 'MAE', 'R2', 'Train Score', 'Test Score'])

# Entraînement et évaluation des modèles par région
for region in regions:
    print(f"Recherche des hyperparamètres optimaux pour la région {region}...")
    
    # Filtrage des données pour la région
    df_region = df_blackout_ml_filtered[df_blackout_ml_filtered['region'] == region]
    
    # Définition des variables explicatives et de la variable cible
    features = df_region[['annee', 'jour_fractionnel_sin', 'jour_fractionnel_cos', 'heure_decimale_sin', 'heure_decimale_cos',
                          'thermique', 'nucleaire', 'eolien', 'solaire', 'hydraulique', 'bioenergies',
                          'pompage', 'stockage_batterie', 'destockage_batterie', 'cap_prod_max_thermique','thermique_p_disp',
                          'cap_prod_max_nucleaire', 'nucleaire_p_disp', 'cap_prod_max_hydraulique', 'hydraulique_p_disp',
                          'cap_prod_max_bioenergies','bioenergies_p_disp',
                          'solaire_p_disp', 'eolien_p_disp', 'population']]    
    
    target = df_region['ech_physiques']
    
    # Division des données en ensembles d'entraînement et de test
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

    # Standardisation des données
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Initialisation du modèle
    xgb = XGBRegressor(random_state=42, objective='reg:squarederror', importance_type='gain')

    # Configuration de la recherche d'hyperparamètres
    random_search = RandomizedSearchCV(estimator=xgb, param_distributions=param_distributions, 
                                       n_iter=50, cv=3, verbose=2, random_state=42, n_jobs=-1)

    # Exécution de la recherche d'hyperparamètres
    random_search.fit(X_train, y_train)
    
    # Enregistrement des meilleurs hyperparamètres
    best_params_region[region] = random_search.best_params_

    # Prédictions avec les meilleurs hyperparamètres
    y_pred = random_search.best_estimator_.predict(X_test)


    # Calcul des métriques
    mse = mean_squared_error(y_test, y_pred)
    rmse = sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    train_score = random_search.best_estimator_.score(X_train, y_train)
    test_score = random_search.best_estimator_.score(X_test, y_test)

    # Création d'un DataFrame temporaire pour les résultats régionaux
    new_result_region = pd.DataFrame({
            'Region': [region],
            'Best Params': [best_params_region[region]],
            'MSE': [mse], 
            'RMSE': [rmse], 
            'MAE': [mae], 
            'R2': [r2],
            'Train Score': [train_score],
            'Test Score': [test_score]
        })
    
    # Ajout des résultats au DataFrame global
    results_region = pd.concat([results_region, new_result_region], ignore_index=True)
    
    print(f"Hyperparamètres optimaux trouvés pour la région {region}. MSE : {mse:.4f}, RMSE: {rmse:.4f}, MAE : {mae:.4f}, R2: {r2:.4f}")

# Exportation des résultats régionaux dans un fichier Excel
results_region.to_excel("hyperpar_region_results_xgboost_bruts.xlsx", index=False, engine='openpyxl')

print("Recherche des hyperparamètres régionaux terminée et résultats sauvegardés.")

Recherche des hyperparamètres optimaux pour la région CENTRE VAL DE LOIRE...
Fitting 3 folds for each of 50 candidates, totalling 150 fits
Hyperparamètres optimaux trouvés pour la région CENTRE VAL DE LOIRE. MSE : 14447.0112, RMSE: 120.1957, MAE : 91.9171, R2: 0.9940
Recherche des hyperparamètres optimaux pour la région PAYS DE LA LOIRE...
Fitting 3 folds for each of 50 candidates, totalling 150 fits
Hyperparamètres optimaux trouvés pour la région PAYS DE LA LOIRE. MSE : 31982.7098, RMSE: 178.8371, MAE : 135.0211, R2: 0.9119
Recherche des hyperparamètres optimaux pour la région GRAND EST...
Fitting 3 folds for each of 50 candidates, totalling 150 fits
Hyperparamètres optimaux trouvés pour la région GRAND EST. MSE : 37261.6979, RMSE: 193.0329, MAE : 151.6654, R2: 0.9874
Recherche des hyperparamètres optimaux pour la région ILE DE FRANCE...
Fitting 3 folds for each of 50 candidates, totalling 150 fits
Hyperparamètres optimaux trouvés pour la région ILE DE FRANCE. MSE : 139688.3955, RMSE: